In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [68]:
# #import sklearn
# from sklearn.cross_validation import train_test_split
# from sklearn import metrics
# from sklearn.cross_validation import cross_val_score
# from sklearn.linear_model import LogisticRegression
#from sklearn import feature_selection as fs #maybe use for p-vals...

In [69]:
# import patsy #not sure if we'll need this

In [9]:
# read the data in
df = pd.read_csv("http://www.ats.ucla.edu/stat/data/binary.csv")

In [10]:
df.head()

,admit,gre,gpa,rank
0,0,380,3.61,3
1,1,660,3.67,3
2,1,800,4.00,1
3,1,640,3.19,4
4,0,520,2.93,4


In [11]:
df.count()

admit    400
gre      400
gpa      400
rank     400
dtype: int64

In [12]:
df.columns[3]

'rank'

In [13]:
df.columns=['admit','gre','gpa','prestige']

In [14]:
df.head()

,admit,gre,gpa,prestige
0,0,380,3.61,3
1,1,660,3.67,3
2,1,800,4.00,1
3,1,640,3.19,4
4,0,520,2.93,4


In [15]:
dummy_ranks = pd.get_dummies(df['prestige'], prefix='prestige') #pd.get_dummies converts a specified categorical variabe into a series of dummy vars
dummy_ranks.head()

,prestige_1,prestige_2,prestige_3,prestige_4
0,0,0,1,0
1,0,0,1,0
2,1,0,0,0
3,0,0,0,1
4,0,0,0,1


In [16]:
#pd.value_counts(df['prestige'])

In [17]:
data = df[['admit','gre','gpa']].join(dummy_ranks.ix[:,'prestige_2':])

In [18]:
data.head()

,admit,gre,gpa,prestige_2,prestige_3,prestige_4
0,0,380,3.61,0,1,0
1,1,660,3.67,0,1,0
2,1,800,4.00,0,0,0
3,1,640,3.19,0,0,1
4,0,520,2.93,0,0,1


In [19]:
train_cols = data.columns[1:]
train_cols

Index(['gre', 'gpa', 'prestige_2', 'prestige_3', 'prestige_4'], dtype='object')

In [20]:
#logit = sm.Logit(data['admit'],data[train_cols]) #How to run a logit with statsmodels.Logit()

In [21]:
#result = logit.fit()

In [22]:
#summary = result.summary()

In [23]:
#summary

In [24]:
#type(summary)

In [25]:
# This is the sklearn way:
# model = LogisticRegression()
# model = model.fit(data[train_cols],data['admit'])
# model.score(data[train_cols],data['admit'])

In [26]:
# data['admit'].mean()

In [27]:
# type(model)

In [28]:
# model

In [29]:
# model.coef_

In [30]:
# train_cols

In [31]:
logit = sm.Logit(data['admit'],data[train_cols])

In [32]:
result = logit.fit()

Optimization terminated successfully.
         Current function value: 0.589306
         Iterations 5


In [33]:
#Use statsmodels over scikit learn for p-vals
logit = sm.Logit(data['admit'],data[train_cols]) #How to run a logit with statsmodels.Logit()
result = logit.fit()
summary = result.summary()
summary

Optimization terminated successfully.
         Current function value: 0.589306
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  admit   No. Observations:                  400
Model:                          Logit   Df Residuals:                      395
Method:                           MLE   Df Model:                            4
Date:                Thu, 07 Apr 2016   Pseudo R-squ.:                 0.05707
Time:                        11:15:37   Log-Likelihood:                -235.72
converged:                       True   LL-Null:                       -249.99
                                        LLR p-value:                 9.732e-06
==============================================================================
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
gre            0.0014      0.001      1.329      0.184        -0.001     0.003
gpa           -0.1336      0.194     -0.687      0.492        -0.515     0.248
prestige_2    -0.9663      0.300     -3.216      0.001        -1.555    -0.377
prestige_3    -1.5451      0.332     -4.648      0.000        -2.197    -0.894
prestige_4    -1.8776      0.402     -4.676      0.000        -2.665    -1.091
==============================================================================
"""

In [34]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 400 entries, 0 to 399
Data columns (total 6 columns):
admit         400 non-null int64
gre           400 non-null int64
gpa           400 non-null float64
prestige_2    400 non-null float64
prestige_3    400 non-null float64
prestige_4    400 non-null float64
dtypes: float64(4), int64(2)
memory usage: 21.9 KB


In [35]:
pd.DataFrame(result.pvalues)

,0
gre,0.183932
gpa,0.492152
prestige_2,0.001298
prestige_3,0.000003
prestige_4,0.000003


In [36]:
numConditions=5
conditionCodes = list(range(numConditions))
conditionCodes

[0, 1, 2, 3, 4]

In [37]:
startCode = np.random.choice(conditionCodes)
startCode

4

In [38]:
assignList = conditionCodes[startCode:] + conditionCodes[:startCode]

In [39]:
assignList = ((len(data)//numConditions)*assignList) + assignList[:(len(data)%numConditions)]

In [40]:
len(assignList)

400

In [41]:
data['condition'] = assignList

In [42]:
#Now try multinomial logit of condition on gre, gpa, prestige_2, prestige_3, prestige_4 
data.head()

,admit,gre,gpa,prestige_2,prestige_3,prestige_4,condition
0,0,380,3.61,0,1,0,4
1,1,660,3.67,0,1,0,0
2,1,800,4.00,0,0,0,1
3,1,640,3.19,0,0,1,2
4,0,520,2.93,0,0,1,3


In [43]:
# # Run logit of treat 1 on control - compare to results from mlogit test
# data_treat1 = data[data['condition']!=2]
# data_treat1.describe()

In [44]:
# logit_treat1 = sm.Logit(data_treat1['condition'],data_treat1[train_cols]) #How to run a logit with statsmodels.Logit()
# result_treat1 = logit_treat1.fit()
# summary_treat1 = result_treat1.summary()
# summary_treat1

In [45]:
# Multinomial Logit with statsmodels -- use when there are more than 2 conditions; works for logit, too :)
mlogit = sm.MNLogit(data['condition'],data[train_cols])
mresult = mlogit.fit()
msummary = mresult.summary()
msummary

Optimization terminated successfully.
         Current function value: 1.602087
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:              condition   No. Observations:                  400
Model:                        MNLogit   Df Residuals:                      380
Method:                           MLE   Df Model:                           16
Date:                Thu, 07 Apr 2016   Pseudo R-squ.:                0.004568
Time:                        11:15:49   Log-Likelihood:                -640.83
converged:                       True   LL-Null:                       -643.78
                                        LLR p-value:                    0.9893
===============================================================================
condition=1       coef    std err          z      P>|z|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------
gre            -0.0010      0.001     -0.693      0.488        -0.004     0.002
gpa             0.1688      0.278      0.608      0.543        -0.375     0.713
prestige_2      0.0058      0.454      0.013      0.990        -0.885     0.897
prestige_3      0.0054      0.488      0.011      0.991        -0.951     0.962
prestige_4      0.2274      0.526      0.433      0.665        -0.803     1.258
-------------------------------------------------------------------------------
condition=2       coef    std err          z      P>|z|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------
gre            -0.0002      0.001     -0.168      0.866        -0.003     0.003
gpa             0.0400      0.279      0.143      0.886        -0.507     0.587
prestige_2      0.0457      0.450      0.102      0.919        -0.836     0.927
prestige_3      0.0059      0.486      0.012      0.990        -0.947     0.958
prestige_4      0.0095      0.537      0.018      0.986        -1.043     1.062
-------------------------------------------------------------------------------
condition=3       coef    std err          z      P>|z|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------
gre            -0.0009      0.001     -0.615      0.538        -0.004     0.002
gpa             0.1793      0.278      0.644      0.520        -0.366     0.725
prestige_2     -0.1998      0.451     -0.443      0.658        -1.084     0.685
prestige_3      0.1116      0.472      0.236      0.813        -0.814     1.037
prestige_4     -0.1571      0.539     -0.291      0.771        -1.214     0.900
-------------------------------------------------------------------------------
condition=4       coef    std err          z      P>|z|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------
gre             0.0008      0.001      0.562      0.574        -0.002     0.004
gpa            -0.2379      0.283     -0.840      0.401        -0.793     0.317
prestige_2      0.2997      0.474      0.632      0.527        -0.630     1.229
prestige_3      0.5768      0.499      1.156      0.248        -0.402     1.555
prestige_4      0.3096      0.556      0.556      0.578        -0.781     1.400
===============================================================================
"""

In [46]:
pVals = mresult.pvalues
pVals
#Columns are labeled by index which is confusion -- correspond to the (column# + 1) condition

,0,1,2,3
gre,0.488177,0.866205,0.538252,0.574442
gpa,0.543008,0.885948,0.519571,0.401020
prestige_2,0.989741,0.919057,0.657908,0.527267
prestige_3,0.991225,0.990348,0.813154,0.247876
prestige_4,0.665294,0.985899,0.770831,0.577960


In [47]:
pVals.columns = conditionCodes[1:] #Rename columns to correspond to conditions
pVals=pVals.reset_index()
pVals.rename(columns={'index':'variable'},inplace=True)

In [48]:
pVals

,variable,1,2,3,4
0,gre,0.488177,0.866205,0.538252,0.574442
1,gpa,0.543008,0.885948,0.519571,0.401020
2,prestige_2,0.989741,0.919057,0.657908,0.527267
3,prestige_3,0.991225,0.990348,0.813154,0.247876
4,prestige_4,0.665294,0.985899,0.770831,0.577960


In [82]:
minPval = .25

In [84]:
#Sample syntax
for index, row in pVals.iterrows():
    row_name = row[0]
    count = 1
    for val in row[1:]:
        if val <= minPval:
            raise Exception('P value for variable {} for condition {} is less than {}.'.format(row_name, count, minPval))
        count += 1

Exception: P value for variable prestige_3 for condition 4 is less than 0.25

In [50]:
#Next: check in any values are <= .05; if they are - randomize again!

In [52]:
pVals.iterrows()

<generator object iterrows at 0x7f1657cbeca8>